# CRM Database Creation Script
This Python script creates a SQLite database named 'account_products.db', which simulates a CRM (Customer Relationship Management) system. The database contains three tables: accounts, products, and account_products. These tables store information about customer accounts, products, and the relationship between accounts and products, respectively. The script uses the Faker library to generate realistic sample data.

## Tables and Columns
**accounts**: This table contains information about customer accounts.

* account_id (TEXT): A unique identifier for the account.
* email (TEXT): The email address associated with the account.
* telephone (TEXT): The phone number associated with the account.
* responsible_name (TEXT): The name of the person responsible for the account.
* address (TEXT): The address associated with the account.
* created_date (DATE): The date when the account was created.

**products**: This table contains information about products.

* product_id (TEXT): A unique identifier for the product.
* product_name (TEXT): The name of the product.
* category (TEXT): The category of the product (e.g., Cat 1, Cat 2).
* monthly_fee (REAL): The monthly fee for the product.

**account_products**: This table establishes the relationship between accounts and products, indicating which products are associated with each account.

* account_id (TEXT): A unique identifier for the account.
* product_id (TEXT): A unique identifier for the product.

## Script Functionality
* generate_product_info: Generates random product information.
* get_created_date: Generates a date between the given start date and the current month.
* generate_account_info: Generates random account information.
* generate_account_product_info: Generates the relationship between an account and products.
* craete_crm_database: Creates the SQLite database with sample data.

The script initializes the Faker library to generate realistic data for accounts and products. It then creates dataframes for products and accounts, including a special account with ID "888-8". The script generates random relationships between accounts and products and writes this data to the SQLite database.

By running this script, you will create a CRM database that can be used for testing and development purposes.

In [1]:
# Create the database and files to use in the example
import sqlite3
from numpy import double
import pandas as pd
import random
import string
from faker import Faker
from datetime import datetime, timedelta


def generate_product_info(fake):
    """ Function to generate random product info. """
    return {
        'product_id': ''.join(random.choices(string.ascii_lowercase + string.digits, k=8)),
        'product_name': fake.word(),
        'category':random.choice(['Cat 1', 'Cat 2', 'Cat 3', 'Cat 4']),
        'monthly_fee': random.randint(0, 100)
    }

def get_created_date(start_date:datetime=None) -> datetime:
    """ Return a date between start_date and the current month.
        If start_adte is None return any date in the last 6 months.
    """
    end_date = datetime.today().replace(day=1)  # First day of the current month
    
    if start_date:
        created_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    else:
       
        created_date = end_date - timedelta(days=random.randint(0, 5*30)) # Any day between the beggining of the month and 6 months before.
    return created_date


def generate_account_info(fake):
    """ Function to generate random account information """

    return {
        'account_id': ''.join(random.choices(string.ascii_lowercase + string.digits, k=8)),
        'email': fake.email(),
        'telephone': fake.phone_number(),
        'responsible_name': fake.name(),
        'address': fake.address(),
        'created_date': get_created_date() 
    }

def generate_account_product_info(account, products_df):
    """ Function to get rellation between account and products"""
    
    return {
        'account_id': account['account_id'],
        'product_id': random.sample(products_df['product_id'].tolist(), 1)[0] 
    }

def craete_crm_database(database_name:str = 'account_products.db', 
                        num_products:int = 100,  
                        num_accounts:int = 50):
    """ Create a SQLite database with two tables: The first with account information and the second with product information"""
    # Initialize Faker
    fake = Faker()

    # Generate products dataframe
    products = []
    for i in range(num_products):
        product_info = generate_product_info(fake)
        products.append(product_info)
    products_df = pd.DataFrame(products)

    # Generate accounts dataframe with associated products and additional info
    accounts = []
    
    ## Add the test account
    accounts.append({'account_id': '888-8',
        'email': fake.email(),
        'telephone': fake.phone_number(),
        'responsible_name': fake.name(),
        'address': fake.address(),
        'created_date': get_created_date() 
        }
    )
    
    ## Add random accounts
    for i in range(num_accounts):
        account_info = generate_account_info(fake)
        accounts.append(account_info)
    accounts_df = pd.DataFrame(accounts)

    # Generate the link between account and products and 
    # create the invoicing

    account_products = []
    for i in range(num_accounts):
        for j in range(random.randint(1, 10)): # Between 1 and 10 products per account
            account_product_info = generate_account_product_info(accounts[i], products_df)
            account_products.append(account_product_info)
           
    account_products_df = pd.DataFrame(account_products)

    # Generate the invoicing 
    
    for i in range(num_accounts):
        for j in range(random.randint(1, 10)): # Between 1 and 10 products per account
            account_product_info = generate_account_product_info(accounts[i], products_df)
            account_products.append(account_product_info)
    account_products_df = pd.DataFrame(account_products)

    # Create a new SQLite database (or connect to an existing one)
    conn = sqlite3.connect(database=database_name)

    # Create tables
    conn.execute('''
    CREATE TABLE IF NOT EXISTS accounts (
        account_id TEXT,
        email TEXT,
        telephone TEXT,
        responsible_name TEXT,
        address TEXT,
        created_date DATE         
    )
    ''')
    conn.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id TEXT,
        product_name TEXT,
        category TEXT,
        monthly_fee REAL         
    )
    ''')

    conn.execute('''
    CREATE TABLE IF NOT EXISTS account_products (
        account_id TEXT,
        product_id TEXT
    )
    ''')


    # Write the data to the database
    accounts_df.to_sql('accounts', conn, if_exists='replace', index=False)
    products_df.to_sql('products', conn, if_exists='replace', index=False)
    account_products_df.to_sql('account_products', conn, if_exists='replace', index=False)

    # Commit and close connection
    conn.commit()
    conn.close()

# Create the CRM Database with Accounts and Products
craete_crm_database()
